# Exploring Materials Synthesis with MP Data and Reaction Networks

**Author:** Matthew McDermott  
**Last updated:** 08/05/21

![](assets/banner.png)

### Goal
*Learn how to build & interpret phase diagrams using MP data and apply these to predict reaction pathways in inorganic materials synthesis.*

### Outline

1. Identifying a target system
2. Building phase diagrams
3. Predicting interface reactions between solids
4. Enumerating all possible reactions to a specific target phase
5. Constructing a reaction network from enumerated reactions
6. Finding and balancing reaction pathways 

## 1. Identifying a target system

**Scenario:** 

While researching ferroelectric materials, you stumble upon a compound that has shown much promise as a multiferroic material: **yttrium manganese oxide, YMnO$_3$**. When you consult the literature, it looks like there are many different approaches for making this material: solid-state synthesis, chimie douce methods, microwave-assisted synthesis, pulsed laser deposition, hydrothermal synthesis, etc. 

In almost all of the papers you find, however, the authors describe significant difficulty in synthesizing phase-pure YMnO$_3$. For example:
- _"... observation that crystallization at low temperature or high oxygen partial pressures gives a mixture of Y2Mn2O7 together with YMnO3" Brinks et al., Journal of Solid State Chemistry 129, 334-340 (1997)._
- _"The solid state synthesis of YMnO3 requires very long procedure with repeated heating and grinding" Z. Brankovic et al. Ceramics International 41 (2015)_
- _"There are still challenges to be addressed with respect to hydrothermal synthesis of h-YMnO3, most notably the difficulty in synthesising phase pure h-YMnO3. This is a result of the complexity of the system, with a total of eight crystalline phases having been detected throughout the course of the reaction." Marshall et al., Chem Eur. J 2020, 26, 9330-9337 (2020)._


Being familiar with the Materials Project database and *pymatgen*, you decide to check the database to see if it can help answer the question: **why is phase-pure YMnO$_3$ so difficult to synthesize?**

We previously learned how to use the *MPRester* to access computed data on the Materials Project. Let's start by importing `MPRester`:

### 1.1 EXERCISE: Acquiring entries from MP
Using the `MPRester`, acquire alll entries from the Materials Project with the composition YMnO$_3$ in the cell below:

### 1.2 Exploring downloaded entries

Great! Now let's take a look at one of the downloaded entry objects:

In the `ComputedStructureEntry` printout, we can see the uncorrected energy, correction, final energy, and the various parameters/data associated with the calculation.

Since this is hard to interpret, it will be easier for us to reformat the entries into a *pandas* `DataFrame` with the relevant information. First we import *pandas*:

And then we create a function to help us wrangle the entries into the `DataFrame` format:

In [ ]:
def get_df_from_entries(entries):
    formulas = [e.composition.reduced_formula for e in entries]
    energies = [e.energy_per_atom for e in entries]
    spacegroups = [e.structure.get_space_group_info()[0] for e in entries]
    structures = [e.structure for e in entries]
    
    data = {"formula": formulas, "energy": energies, "spacegroup": spacegroups, "entry":entries}
    
    df = pandas.DataFrame(data).sort_values("energy")
    
    return df

The dataframe can now be created by calling the function:

 We see that there are several polymorphs of similar energy. The lowest energy polymorph is the hexagonal YMnO$_3$ phase which is well known as the stable phase in the literature:

If we want to interactively view this structure in JupyterLab, we can also import crystal_toolkit to view it:

## 2. Building phase diagrams

Now that we've confirmed the existence of the target phase within the MP database, we'd like to better understand phase composition within the Y-Mn-O system. This means answering questions suchs as:

1. Just how stable is YMnO$_3$? 
2. How much energy is released upon formation of YMnO$_3$ from binary oxides vs. the elements?
3. What other phases might compete against the formation of YMnO$_3$? 
4. What kinds of oxygen chemical potentials stabilize YMnO$_3$? 

These are all questions that can be answered by constructing phase diagrams within *pymatgen*.

### 2.1 EXERCISE: Acquiring all entries within the chemical system
Let's start by downloading entries for the full Y-Mn-O system using `MPRester`. How many entries exist within the full Y-Mn-O system?

### 2.2 Compositional phase diagrams
Now that we have all the entrise within the Y-Mn-O system, we can create a ternary compositional phase diagram by simply passing the list of entries to create a `PhaseDiagram` object. Let's first import the phase_diagram module:

And create the phase diagram:

To plot the phase diagram, we can either:
1. Create a `PDPlotter` object and call `get_plot()` with custom arguments
2. Create a `PDPlotter` object and call `show()`
3. In JupyterLab, just type the name of the phase diagram object in a cell and click Enter

Let's do option 1. First we import `PDPlotter`:

And then we create the plotting object and call the `get_plot` function:

The default backend for plotting phase diagrams is *plotly*, however, you can also specify *matplotlib*:

From the phase diagram, we see that there are three ternary oxides within the system: YMnO$_3$, YMn$_2$O$_5$, and Y$_2$Mn$_2$O$_7$. 

Let's look at formation energy data in a *pandas* `DataFrame`. As before, we create a function to wrangle the data into the correct format:

In [ ]:
def get_df_from_pd(pd):
    ents = pd.stable_entries
    
    formulas = [e.composition.reduced_formula for e in ents]
    form_energies = [pd.get_form_energy_per_atom(e) for e in ents]
    decomp_enthalpies = [pd.get_phase_separation_energy(e) for e in ents]
    
    data = {"formula": formulas, "form_energy (eV/atom)": form_energies, "decomp_enthalpy (eV/atom)": decomp_enthalpies}
    
    df = pandas.DataFrame(data).sort_values("form_energy (eV/atom)").reset_index(drop=True)
    
    return df

We then call this function to get the data frame:

While YMnO$_3$ has one of the lowest formation energies within the Y-Mn-O system, it's decomposition enthalpy (energy "below" hull) is not very negative compared to the other phases in the system. This suggests that it has a low relative stability compared to the neighboring phases.

This is largely due to the fact that **Y$_2$O$_3$ is a massive thermodynamic sink** -- it has both the most negative formation energy and the most negative decomposition enthalpy. We can even see this on the color-shading of the ternary phase diagram.


### 2.3 Compound phase diagrams for plotting subsets of the convex hull

The previous analysis partially explains why synthesis routes to YMnO$_3$ from the starting binary oxides (i.e. Y$_2$O$_3$ and Mn$_2$O$_3$) require such high temperatures to proceed. We can take a slice of the hull with the `CompoundPhaseDiagram` to visualize this. First, let's import the module:

We also have to specify the terminal compositions as `Composition` objects within pymatgen:

Now we can initialize the compound phase diagram object and plot it:

From the plot, if we hover over YMnO$_3$, we see that the reaction energy between Y$_2$O$_3$-Mn$_2$O$_3$ is calculated to be $\Delta$H = -0.044 eV/atom.

An alternative way to derive this is by providing the entries to the `ComputedReaction` object, which will automatically balance the reaction for us. Let's import it:

We next have to acquire the entries by specifying the formulas:

We can then initialize the `ComputedReaction` object:

**Note:** The reaction energy associated with the ComputedReaction object is eV per mole of reaction (i.e. per mole of YMnO$_3$):

However, if we calculate the number of atoms in the reaction we can normalize this to a a **reaction energy per atom** basis:

And now we see that the -0.044 eV/atom reaction energy matches with the result from the `CompoundPhaseDiagram` interface.

Note that the `CompoundPhaseDiagram` is also useful for plotting subsections of the convex hull:

### 2.4 Incorporating temperature-dependence with a previously derived ML model ($G_{SISSO}$) and expt data

The Materials Project contains DFT-calculated energies for commpounds at T=0 K. Since most experimental syntheses take place well above 0 K, it is useful to have some idea of how the phase diagram (and hence reaction energies, relative stabilities, etc.) change with increasing temperature.

A recent work by Bartel et al. derived an equation to estimate the Gibbs free energy of formation of solids, $\Delta G_f(T)$, as a function of temperature using the MP formation enthalpy (T=298 K) as a reference. This has been implemented within pymatgen as the `GibbsComputedStructureEntry` class. Note that this implementation also includes the change in elemental chemical potentials as a function of temperature, as well as as the thermochemistry data for some known gases such as CO$_2$.

**Reference**: 
##### _Bartel, C. J., Millican, S. L., Deml, A. M., Rumptz, J. R., Tumas, W., Weimer, A. W., … Holder, A. M. (2018). Physical descriptor for the Gibbs energy of inorganic crystalline solids and temperature-dependent materials chemistry. Nature Communications, 9(1), 4168. https://doi.org/10.1038/s41467-018-06682-4_

We can create the phase diagram at T = 2000 K and compare this to our previous MP (T=298 K) phase diagram. First let's import the module:

Since the entries require phase diagram derived data, it is most convenient to initialize a list of them from a list of regular `ComputedStructureEntry` objects, as below:

Let's print one of the entries to see what it looks like:

Now we can make and plot our phase diagram with the new Gibbs entries:

We see that Mn$_2$O$_3$, MnO$_2$, and Y$_2$Mn$_2$O$_7$ are all predicted to become significantly destabilized at higher temperatures. While some of this behavior could arise from differences in solid-state entropies, this is most likely a result of the significant decrease in oxygen chemical potential at high temperatures, which creates more reducing conditions that favor phases containing less oxygen. 

### 2.5 Grand potential phase diagrams for open elements
In solid-state synthesis, materials are often synthesized in some type of gaseous environment. This may be an inert gas, flowing/controlled oxygen, etc. A common question in materials synthesis is: **what kind of atmosphere do I need to promote the synthesis of my target material?**

For oxides such as YMnO$_3$ this means determining how oxidizing/reducing the environment should be: i.e. determining the partial pressure (and hence chemical potential) of oxygen.

In thermodynamics, a system open to an element can be modeled with the grand potential, $\phi$. For open oxygen, this corresponds to:

$$\phi = G - \mu_On_O$$

We can switch to the grand potential by using the `GrandPotentialPhaseDiagram` within *pymatgen*. From now on, we will use `GibbsComputedStructureEntry` objects, as they are referenced to a standard state where elemental chemical potentials are 0 eV. This helps us better understand chemical potentials as a function of formation energies (rather than DFT energies).

From the grand potential phase diagram above, we see that only YMn$_2$O$_5$ and Y$_2$Mn$_2$O$_7$ are stable at a relative oxygen chemical potential of 0 eV at T = 1000 K. This corresponds to flowing (open) oxygen gas at the standard state of pressure of 0.1 MPa (close to 1 atm).

Note that the is for a binary system, since we have removed a degree of freedom in fixing the oxygen chemical potential to a specific value.

#### 2.5.2 EXERCISE: Determine oxygen stability window of YMnO$_3$

Determine the partial pressure of oxygen (T = 1000 K) where YMnO$_3$ becomes stable.

**Hint:** *Determine the chemical potential of oxygen and use the following formula to solve for the pressure:*

$$ \mu - \mu^0 = k_b T \ln{\frac{p}{p_0}} $$

In [ ]:
from math import exp
k = 8.617e-5  # eV/K
T = 1000  # K

mu_O = _
mu_O2 = mu_O * 2  

...

This is quite a low O$_2$ partial pressure. However, note that this value is extremely sensitive to small differences in calculated formation energies for the compounds. 

Temperature is often an even more powerful driver for oxidizing/reducing conditions than pressure. In fact, at higher temperatures (T = 1400 K), YMnO3 is predicted to be stable even at the standard state (0.1 MPa) pressure.

#### 2.5.3 Finding "critical" chemical potentials within pymatgen

As was hinted at above, all phases on the compositional phase diagram are stable within a certain region of chemical potentials. These do not need to be found manually as above, instead there are methods to solve for them algorithmically. For YMnO$_3$, we get:

Note that the upper bound value of chemical potential for oxygen matches what we found in the previous exercise (-0.343 eV).

### 2.6 Plotting predominance diagrams

The chemical potential ranges over which a phase is stable can be plotted in a new diagram, historically called a predominance diagram. Because one chemical potential is always dependent, we only need 2 dimensions to represent the chemical potential stability regions. These form polygons, which are not necessarily rectangular!

**Note:** the -0.343 eV value where YMnO$_3$ becomes stable is actually the tip of the narrow YMnO$_3$ polygon!

### 2.7 BONUS: Plotting chemical potential diagrams (adding one more dimension!)

It is often even more convenient to visualize all three dimensions within chemical potential space. For this, we can plot a **chemical potential diagram**. For now, this code only exists in the (separate) reaction-network package, but will be added soon to pymatgen.

In [ ]:
from rxn_network.thermo.chempot_diagram import ChempotDiagram

To create the diagram, we first need to define some limits on the chemical potential ranges to plot:

In [ ]:
limits={Element("Y"): (-10, 0),
        Element("Mn"): (-10, 0),
        Element("O"): (-7, 0)
       }

And then we can plot it similar to plotting a conventional phase diagram in pymatgen:

In [ ]:
cd = ChempotDiagram(pd_gibbs, limits=limits)
cd.get_plot()

From the chemical potential diagram, we can easily see the relative stabilities of each phase and get a visual picture of where the critical chemical potentials are for each phase.

## 3. Predicting interface reactions between solids

### 3.1 A simple model of solid-state synthesis

In solid-state synthesis, precursor compounds are typically ground/milled into powders of small solid crystallites. These powders are intimately mixed, typically pressed into a pellet, and then heated in a furnace. Typically, solid-state synthesis involves high temperatures (i.e. 1200 ºC) and long heating times (24-72 hrs). However, many reactions can proceed significantly faster (on the order of a few minutes) and at temperatures as low as 400-500ºC. 

A simple cartoon of solid-state synthesis can be seen below:

<img src="assets/interface_rxn.png" width="750">

At the interfaces where solid crystallites touch, if the free energy of the system can be decreased by forming a new phase or set of phases, the new phase(s) will nucleate and grow.

We can predict the possible reactions between the interface of any two phases by drawing a line (see the red dashed line below) connecting the two phases on the phase diagram. A reaction is given by each point where the connecting line intersects the phase diagram points/lines. 

In the hypothetical system below, there are two predicted reactions given by the yellow star and green circle -- in this case **the reaction to form $\gamma$ (green circle) is the most thermodynamically favorable** because it involves the greatest decrease in free energy. However, the reaction given by the yellow star is still favorable and may occur at higher temperatures / later times.

<img src="assets/interface_pd.png" width="900">

### 3.2 Using pymatgen's interface reaction calculator

The phase diagram slice approach depicted above has been implemented into pymatgen within the `InterfacialReactivity` class and is available as an app on the MP website. 

Let's see if we can apply it within the Y-Mn-O chemical system to better understand the synthesis of our YMnO$_3$ target. We will start with plotting the suggested reactions between Y$_2$O$_3$ and Mn$_2$O$_3$:

In the plot, we see a new convex hull slice, where each point represents a reaction as a function of the (normalized) ratio of mixing between the two reactants. To see which reactions the plot above corresponds to, we need to extract data about the "kinks" within the diagram:

In [ ]:
pandas.set_option("max_colwidth", 80)  # make columns wider to see full reaction

def get_df_from_interface_rxns(ir):
    critical_rxns = [
        {"Atomic fraction":round(ratio, 3),
         "Reaction": rxn,
            "E$_{rxn}$ (kJ/mol)": round(rxn_energy, 1),
            "E$_{rxn}$ (eV/atom)":round(reactivity, 3)
        }
        for _, ratio, reactivity, rxn, rxn_energy in ir.get_kinks()
    ]
    
    ir_df = pandas.DataFrame(critical_rxns)
    return ir_df

In the table, we see the reactions as a function of the normalized mixing ratio, along with their energy in kJ per mole of reaction, as well as eV per (reactant) atom. 

It seems that YMnO$_3$ is indeed the major predicted reaction product when Y$_2$O$_3$ and and Mn$_2$O$_3$ are combined in equal molar amounts. However, the reaction forming YMn$_2$O$_5$ and Mn$_3$O$_4$ could still occur to some degree depending on the process by which the synthesis procedure occurs (i.e. initial grain sizes, heating rate, maximum temperature reached, etc.) This is because **there is still a thermodynamic driving force for subsequent interfacial reaction between YMnO$_3$ and any remaining precursor Mn$_2$O$_3$**:

It turns out that this is true again for any Mn$_3$O$_4$ produced, which may react with remaining Y$_2$O$_3$ precursor to make YMnO$_3$ and MnO. The same goes again for MnO and Mn$_2$O$_3$. 

Just in this simple ternary system, we can already see the **complex (and sometimes unpredictable) behavior associated with reaction pathways within solid-state synthesiss**. To organize this progression of reactions, we choose to create a graph network representation of the possible reaction pathway, such as the one illustrated below:

<img src="assets/small_rxn_network.png" width="900">

Before we can explore this reaction network structure further, however, we need to first discuss how to acquire all possible reactions which may occur within a chemical system.

## 4. Enumerating all reactions within a chemical system

As we saw above, the possibility of subsequent reactions makes a model for solid-state synthesis quite complex. If we wish to capture this behavior, we first need to be able to enumerate through all possible reactions within a chemical system. These reactions will then be used to construct the reaction network.

There are two methods for predicting reactions in a high-throughput manner: 

1) interface reactions iteration through all possible slices of the full phase diagram (as above)
2) brute force trial & error balancing between set of reactants and any set of products

Both of these methods are implemented in the *reaction-network* package, which builds reaction network analysis on top of pymatgen:

<img src="assets/logo.png" width="700">

### 4.1 Minimize Gibbs (interface reaction) approach

First, we need to come up with a candidate set of entries. Since we are using the reaction-network package, we are going to initialize the entries using that package's convenience methods:

Now, we can initialize the enumerator object:

To enumerate reactions, we call `enumerate()` with a list of provided entries. This will return a list of reactions which are computed using the interface reaction method from above:

As before, let's look at these in a DataFrame:

In [ ]:
def get_df_from_rxns(rxns):
    energies = [r.energy_per_atom for r in rxns]
    uncertainties = [r.energy_uncertainty_per_atom.s for r in rxns]
    chemsyses = [r.chemical_system for r in rxns]
    
    data = {"reaction": rxns, "energy": energies, "dE": uncertainties, "chemsys": chemsyses}
    
    return pandas.DataFrame(data).sort_values("energy").reset_index(drop=True)

We can plot a histogram of the reaction energies to get an idea of the thermodynamics of the system. Again, we see that Y$_2$O$_3$ is quite stable (the reaction on the far left!). Most reactions, however, are in the -0.4 to 0.0 eV/atom range.

### 4.2 Brute force (simple) approach

Enumerating reactions by brute force may seem like a less advanced approach, but it offers the added benefit that we can identify reactions involving:

1) metastable products
2) products which are not stable with respect to each other

This approach has been implemented with the `BasicEnumerator` class in the reaction-network package:

Let's try using entries up to an energy cutoff of 30 meV/atom above the hull:

Now we will enumerate via a brute force approach all reactions within this set of expanded entries. Note that we get both the forward and reverse reactions from this approach, producing a symmetric reaction energy distribution:

### 4.3 Enumerating reactions to a specific target or from specific precursors
The Enumerator classes also allow for specifying a desired target phase:

It is also possible to specify precursors:

Or even specify both precursors *and* target:

### 4.4 BONUS: Enumerating open reactions

Since solid-state synthesis often happens in the presence of some kind of gaseous or "open" environment, it is also possible to enumerate so-called open reactions where a particular entry is open.

In [ ]:
from rxn_network.enumerators.basic import BasicOpenEnumerator

In [ ]:
boe = BasicOpenEnumerator(["O2"])
open_rxns = boe.enumerate(entries_030)

In [ ]:
get_df_from_rxns(open_rxns)

If these reactions involve an open entry that is also an element, then it is possible to recalculate the reaction energy in terms of grand potential:

In [ ]:
from rxn_network.reactions.reaction_set import ReactionSet

In [ ]:
rxn_set = ReactionSet.from_rxns(open_rxns)
open_rxns_grand = rxn_set.get_rxns(open_elem=Element("O"), chempot=0.0)

In [ ]:
get_df_from_rxns(open_rxns_grand)

Similarly, it is also possible to enumerate reactions straight from the grand potential phase diagram:

In [ ]:
from rxn_network.enumerators.minimize import MinimizeGrandPotentialEnumerator

mgpe = MinimizeGrandPotentialEnumerator(open_elem=Element("O"), mu=0.0)
open_rxns2 = mgpe.enumerate(entries_030)

In [ ]:
get_df_from_rxns(open_rxns2)

## 5. Constructing a reaction network from enumerated reactions

Now that we have a way to enumerate all reactions within a chemical system, we'd like to be able to put them in a convenient data structure such that we can learn something. One particular data structure that is helpful for predicting reaction pathways is a **weighted directed graph**. 

The weighted directed graph, or reaction network, connects all phases within the chemical system via the possible reactions between them. This corresponds to a structure like the following:
- **Nodes**: Phase combinations
- **Edges**: Reactions
- **Edge weights/costs**: Reaction free energies (after monotonic transformation)

The function that monotonically transforms reaction free energies to (positive) costs/weights is dubbed a cost function.

For more information about this approach, please see the following publication:

**Reference:** *McDermott, M. J., Dwaraknath, S. S., and Persson, K. A. (2021). A graph-based network for predicting chemical reaction pathways in solid-state materials synthesis. Nature Communications, 12(1). https://doi.org/10.1038/s41467-021-23339-x*

Let's start by reacquiring entries for the Y-Mn-O system and filtering for stable entries.

Now we specify the enumerator(s) and cost function to use in the network construction. At the moment, only the softplus function is implemented.

We need to import the `logging` package to get information about the network creation process:

We then initialize our reaction network object:

The network can be built by calling `build()`. This runs through each enumerator provided, and constructs the reaction network structure within the _graph-tool_ package. The graph object is stored under the `graph` attribute.

To perform pathfinding, we first need to set the precursors and target defining the particular synthesis procedure we are interested in. For now, let's look at the synthesis of YMnO3 from the starting binary oxides, Y2O3 and Mn2O3. We first acquire the entries from the set of entries by providing the formula:

And then we set the precursors and target nodes:

We can also draw a cartoon illustration of what this reaction network looks like:

We can perform simple pathfinding between the precursors node and target node. This finds the k-shortest paths between our precursors and target, which yields a sequence of reaction steps. **Note: these are not necessarily mass balanced!**

## 6. BONUS: Finding and balancing reaction pathways from a real experiment

In 2019, Todd et al. reported the synthesis of YMnO3 through an "assisted metathesis" approach corresponding to the following reaction equation:

$$ Mn_2O_3 + 2 YCl_3 + 3 Li_2CO_3 \rightarrow 2 YMnO_3 + 6 LiCl + 3 CO_2 $$

They also observed the following progression of phases, as determined via in situ X-ray diffraction on a synchrotron beamline:

<img src="assets/ymno3.png" width="500">

**Reference:** *Todd, P. K., Smith, A. M., &amp; Neilson, J. R. (2019). Yttrium manganese oxide phase stability and selectivity using lithium carbonate assisted metathesis reactions. Inorganic Chemistry, 58(22), 15166–15174. https://doi.org/10.1021/acs.inorgchem.9b02075*

To model this synthesis, we first acquire the entries for the entire 6-element chemical system from MP:

In [ ]:
with MPRester() as mpr:
    entries = mpr.get_entries_in_chemsys("Y-Mn-O-Li-Cl-C")

And we filter this entries by a stability filter of 20 meV/atom at T = 650 ºC:

In [ ]:
temp = 650 + 273
gibbs_entries = GibbsEntrySet.from_entries(entries, temperature=temp).filter_by_stability(0.020)

We then designate the enumerator, cost function, and build the network. This may take a minute!

In [ ]:
be = BasicEnumerator()
cf = Softplus(temp=temp)

In [ ]:
rn = ReactionNetwork(gibbs_entries, [be], cf)

In [ ]:
rn.build()

As before, we then set the precursors:

In [ ]:
rn.set_precursors([gibbs_entries.get_min_entry_by_formula("Li2CO3"), 
                   gibbs_entries.get_min_entry_by_formula("Mn2O3"), 
                   gibbs_entries.get_min_entry_by_formula("YCl3")])

And then we perform pathfinding to all three targets simultaneously:

In [ ]:
paths = rn.find_pathways(["YMnO3","LiCl","CO2"], k=10)

Finally, we must enforce mass balance to filter out unfeasbile pathways. To do this, we import and use the `PathwaySolver` class:

In [ ]:
from rxn_network.pathways.solver import PathwaySolver
from rxn_network.reactions.computed import ComputedReaction

ps = PathwaySolver(rn.entries, paths, Softplus(1000))

The `PathwaySolver` takes a net reaction by which to enforce mass balance, i.e. total consumption of all produced intermediates. This corresponds to the previous reaction formula we showed. We can create a `ComputedReaction` object by finding the relevant entries and providing them to the reaction constructor:

In [ ]:
product_entries = []

for i in ["YMnO3", "LiCl", "CO2"]:
    product_entries.append(gibbs_entries.get_min_entry_by_formula(i))

In [ ]:
net_rxn = ComputedReaction.balance(rn.precursors,product_entries)
print(net_rxn)

Finally, we solve for the balanced paths and print them. This may take a minute!

In [ ]:
balanced_paths = ps.solve(net_rxn)

The final balanced pathways can be printed, as below:

In [ ]:
for p in balanced_paths:
    print(p, "\n")

As we see, the balanced pathways reported capture many of the experimentally observed intermediates and reaction steps!

For more information or questions regarding this lesson, please contact **mcdermott [at] lbl.gov** Thank you!